# カタラン数から円周率を求める

カタラン数とは、整数 $n \geq 0$ に対して定まる自然数 $C_n$ であって

$$C_0 = 1, \quad C_n = \sum_{k=1}^{n} C_{k-1} C_{n-k}$$

を満たすもののことです。例えば

$$\begin{align*}
C_1 &= C_0 C_0 = 1, \\
C_2 &= C_0C_1 + C_1C_0 = 2,\\
C_3 &= C_0 C_2 + C_1C_1+ C_2 C_0 = 5
\end{align*}$$

です。カタラン数は

$$\pi = \lim_{n \to \infty} \frac{16^{n}}{n^3C_n^2}$$

を満たします。これを計算してみましょう。

In [3]:
from functools import lru_cache

@lru_cache(maxsize=10**3*5)
def catalan(n: int) -> int:
    if n == 0:
        return 1
    else:
        l_ = [catalan(k-1) for k in range(1, n+1)]
        return sum([
            c * cr for (c, cr) in zip(l_, reversed(l_))
        ])

    
def calc_pi_1(n: int) -> float:
    return (4 ** (2 * n)) / (n ** 3 * catalan(n) ** 2)

def calc_pi_2(n: int, catalan_num: int) -> float:
    return (4 ** (2 * n)) / (n ** 3 * catalan_num ** 2)

for n in [10, 10 **2, 10 ** 3]:
    print(f"n = {n: <6} \t {catalan(n)}")

n = 10     	 16796
n = 100    	 896519947090131496687170070074100632420837521538745909320
n = 1000   	 2046105521468021692642519982997827217179245642339057975844538099572176010191891863964968026156453752449015750569428595097318163634370154637380666882886375203359653243390929717431080443509007504772912973142253209352126946839844796747697638537600100637918819326569730982083021538057087711176285777909275869648636874856805956580057673173655666887003493944650164153396910927037406301799052584663611016897272893305532116292143271037140718751625839812072682464343153792956281748582435751481498598087586998603921577523657477775758899987954012641033870640665444651660246024318184109046864244732001962029120


In [4]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import color_pi, PI_50


for n in [10, 10**2, 10**3, 10**3 * 5]:
    print(f"n = {n: <6} \t {color_pi(str(calc_pi_1(n)), PI_50)}")

n = 10     	 3.8975176863405654
n = 100    	 3.2127605022857373
n = 1000   	 3.1486660485813665
n = 5000   	 3.1430065627141164


## 円周率の計算の精密化

カタラン数 $C_n$ は

$$C_n = \frac{1}{n+1} \binom{2n}{n}$$

が成り立つことが知られています。　これとウォリスの公式

$$\begin{gather*}
W_n = \frac{(2n)!!^2}{(2n-1)!!(2n+1)!!} \\
2 W_n \leq \pi \leq \frac{2n+1}{n} W_n \\
\lim_{n\to\infty} W_n = \frac{\pi}{2}
\end{gather*}$$

を合わせて円周率の計算を精密化します。カタラン数を計算すると

$$\begin{align}
C_n &= \frac{1}{n+1} \binom{2n}{n} = \frac{1}{n+1} \frac{(2n)!}{n! n!} = \frac{1}{n+1} \frac{(2n-1)!! (2n)!!}{n! n!} \\
&= \frac{1}{n+1} \frac{(2n-1)!! (2^n n!)}{n! n!} = \frac{4^{n}}{n+1} \frac{(2n-1)!!}{(2n)!!} \\
&= \frac{4^{n}}{n+1} \sqrt{\frac{1}{\frac{(2n)!!^2}{(2n-1)!!^2 (2n+1)}(2n+1)}} \\
&= \frac{4^{n}}{(n+1)\sqrt{2n+1}} \frac{1}{\sqrt{W_n}}
\end{align}
$$

となります。ウォリスの公式から

$$\begin{align}
\frac{2 \cdot 16^n}{(n+1)^2(2n+1)C_n^2} \leq \pi & \leq \frac{2n+1}{n} \frac{16^n}{(n+1)^2(2n+1)C_n^2} \\
& = \frac{16^n}{n(n+1)^2C_n^2} 
\end{align}
$$

となり、円周率を上下から評価できます。覚えにくいので、$\frac{2}{2n+1} > \frac{1}{n+1}$, $\frac{1}{n + 1} < \frac{1}{n}$ を用いて

$$\begin{align}
\frac{16^n}{(n+1)^3C_n^2} \leq \frac{2 \cdot 16^n}{(n+1)^2(2n+1)C_n^2} \leq \pi \leq \frac{16^n}{n(n+1)^2C_n^2}  \leq \frac{16^n}{n^3C_n^2} 
\end{align}
$$

とすると、収束は少し遅くなりますがすっきりします。

また、カタラン数を定義通りの漸化式を用いて計算すると計算リソースを多く必要としますが、

$$a_n = \frac{16^n}{\binom{2n}{n}^2}$$

とおけば、

$$\frac{2}{2n+1} a_n \leq \pi \leq \frac{1}{n}a_n$$

であり、$a_1 = 4$ かつ

$$\begin{align*}
a_{n+1} &= \frac{16^{n+1}}{\binom{2n + 2}{n + 1}^2} \\
&= 16 \cdot 16^n \cdot \frac{(n+1)!^4}{(2n+2)!^2} \\
&= 16^n \cdot \frac{n!^4}{(2n)!^2} \cdot 16 \cdot \frac{(n+1)^4}{(2n+2)^2(2n+1)^2} \\
&= a_n \cdot \frac{4(n+1)^2}{(2n+1)^2} \\
&= a_n \left(1 +\frac{1}{2n+1}\right)^2
\end{align*}$$

により $a_n$ を計算することができます。こちらの方が桁が大きくなりすぎず、計算しやすいです。

In [5]:
from typing import Generator, Tuple
import mpmath

def an_() -> Generator[Tuple[float, int], None, None]:
    n = 1
    val = mpmath.mpf(4)
    
    while True:
        yield val, n
        val = val * (1 + 1 /(2 * n+1)) ** 2
        n = n + 1

$$\frac{2 a_n}{2n+1} = \frac{2 \cdot 16^n}{(n+1)^2(2n+1)C_n^2} \leq \pi \leq \frac{16^n}{n(n+1)^2C_n^2} = \frac{a_n}{n} $$

In [6]:
DPS = 25
mpmath.mp.dps = DPS

an = an_()


print_time = [10, 10 ** 2, 10**3, 10 ** 4, 10 ** 5, 10 ** 6]
for val, n in an:
    if n in print_time:
        low_pi = 2 * val / (2 * n + 1)
        up_pi = val / n
        
        low_str = mpmath.nstr(low_pi, 10, strip_zeros=False)
        up_str = mpmath.nstr(up_pi, 10, strip_zeros=False)
        
        print(f"n = {n: <14} \t {color_pi(low_str, PI_50)} ≦ π ≦ {color_pi(up_str, PI_50)}")
    if n > print_time[-1]:
        break

n = 10             	 3.067703807 ≦ π ≦ 3.221088997
n = 100            	 3.133787491 ≦ π ≦ 3.149456428
n = 1000           	 3.140807746 ≦ π ≦ 3.142378150
n = 10000          	 3.141514119 ≦ π ≦ 3.141671194
n = 100000         	 3.141584800 ≦ π ≦ 3.141600508
n = 1000000        	 3.141591868 ≦ π ≦ 3.141593439


$$\frac{a_n}{n+1}= \frac{16^n}{(n+1)^3C_n^2} \leq \pi \leq \frac{16^n}{n^3C_n^2} = \left(\frac{n+1}{n}\right)^2 \frac{a_n}{n} $$

In [7]:
DPS = 25
mpmath.mp.dps = DPS

an = an_()


print_time = [10, 10 ** 2, 10**3, 10 ** 4, 10 ** 5, 10 ** 6]
for val, n in an:
    if n in print_time:
        low_pi = val / (n+1)
        up_pi = ((n+1)/n) **2 * val / n
        
        low_str = mpmath.nstr(low_pi, 10, strip_zeros=False)
        up_str = mpmath.nstr(up_pi, 10, strip_zeros=False)
        
        print(f"n = {n: <14} \t {color_pi(low_str, PI_50)} ≦ π ≦ {color_pi(up_str, PI_50)}")
    if n > print_time[-1]:
        break

n = 10             	 2.928262725 ≦ π ≦ 3.897517686
n = 100            	 3.118273691 ≦ π ≦ 3.212760502
n = 1000           	 3.139238911 ≦ π ≦ 3.148666049
n = 10000          	 3.141357059 ≦ π ≦ 3.142299560
n = 100000         	 3.141569092 ≦ π ≦ 3.141663340
n = 1000000        	 3.141590297 ≦ π ≦ 3.141599722
